In [1]:
import numpy as np
from NeuralNetwork import NeuralNetwork
from load import *
%load_ext autoreload
%autoreload 2

In [2]:
X_train, y_train = load_mnist('./data')
X_test, y_test = load_mnist('./data', 't10k')

In [13]:
print(X_train.shape)
print(y_train.shape)

(60000, 784)
(60000,)


In [3]:
parameters = {
            'layer_sizes': [X_train.shape[1], 80, 40, 10],
            'learning_rate': 0.001,
            'decay_rate': 0.00001,
            'l2': 0.1,
            'momentum_const': 0.5,
            'minibatch': 500,
            'epochs': 2000,
            'dropout': True,
            'dropout_rate': 0.5,
        }
nn = NeuralNetwork(**parameters)
y_enc = nn.one_hot(y_train, 10)

In [4]:
print(len(nn.weights))
for i in range(len(nn.weights)):
    print(nn.weights[i].shape)

3
(785, 80)
(81, 40)
(41, 10)


In [5]:
activations, zs = nn.forwardProp(X_train, dropout=True)

activations.py:29: RuntimeWarning: overflow encountered in exp
  d = 1.0/(1.0 + np.exp(-w))


In [6]:
print(len(activations))
print(len(zs))
for i in range(len(activations)):
    print(activations[i].shape)
for i in range(len(zs)):
    print(zs[i].shape)

4
3
(60000, 785)
(60000, 81)
(60000, 41)
(60000, 10)
(60000, 80)
(60000, 40)
(60000, 10)


In [7]:
grads = nn.backProp(activations, zs, y_enc)

In [8]:
print(len(grads))
for i in range(len(grads)):
    print(grads[i].shape)

3
(785, 80)
(81, 40)
(41, 10)


In [9]:
cost = nn.get_cost(y_enc, output=activations[-1])
print(cost)

0.892588463615


In [10]:
nn.fit(X_train, y_train, print_progress=True, check_gradients=False)
accuracy = nn.accuracy(X_test, y_test)
print("Accuracy:", accuracy)

Epoch: 50
Loss: 2.61844054059
Training Accuracy: 84.0733333333
Epoch: 100
Loss: 2.57388167917
Training Accuracy: 86.495
Epoch: 150
Loss: 2.47113728474
Training Accuracy: 86.6716666667
Epoch: 200
Loss: 2.29279584004
Training Accuracy: 86.3283333333
Epoch: 250
Loss: 2.47104520242
Training Accuracy: 88.0483333333
Epoch: 300
Loss: 2.48415472521
Training Accuracy: 87.7883333333
Epoch: 350
Loss: 2.27926648898
Training Accuracy: 88.355
Epoch: 400
Loss: 2.22231508787
Training Accuracy: 88.63
Epoch: 450
Loss: 2.23570268585
Training Accuracy: 88.9383333333
Epoch: 500
Loss: 1.8964942739
Training Accuracy: 89.3183333333
Epoch: 550
Loss: 1.98277295268
Training Accuracy: 89.5116666667
Epoch: 600
Loss: 1.96784553402
Training Accuracy: 90.0183333333
Epoch: 650
Loss: 1.78928697919
Training Accuracy: 90.2583333333
Epoch: 700
Loss: 1.81451729284
Training Accuracy: 90.55
Epoch: 750
Loss: 1.51959736368
Training Accuracy: 90.72
Epoch: 800
Loss: 1.66047243798
Training Accuracy: 91.0416666667
Epoch: 850
Loss:

KeyboardInterrupt: 

In [11]:
accuracy = nn.accuracy(X_test, y_test)
print("Accuracy:", accuracy)

('Accuracy:', 91.81)


In [23]:
X = np.array([[0,0],[1,0],[0,1],[1,1]])
y = np.array([1,0,0,1])
y_hot = nn.one_hot(y, 2).astype('int64')
print(X.shape)
print(y.shape)
print(y_hot.shape)

(4, 2)
(4,)
(4, 2)


In [24]:
y_hot


array([[0, 1],
       [1, 0],
       [1, 0],
       [0, 1]])

In [27]:
parameters = {
            'layer_sizes': [X.shape[1], 4, 2],
            'learning_rate': 0.001,
            'decay_rate': 0.00001,
            'l2': 0.1,
            'momentum_const': 0.5,
            'minibatch': 500,
            'epochs': 2000,
            'dropout': True,
            'dropout_rate': 0.5,
        }
nn = NeuralNetwork(**parameters)
nn.fit(X, y_hot, print_progress=True, check_gradients=False)

Epoch: 50
Loss: inf


ValueError: operands could not be broadcast together with shapes (4,) (4,2) 

# Experiments

In [ ]:
sizes = [784, 40, 10]
biases = [np.random.randn(y, 1) for y in sizes[1:]]

In [ ]:
len(biases)

In [ ]:
print(biases[0].shape)
print(biases[1].shape)

In [ ]:
weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

In [ ]:
print(weights[0].shape)
print(weights[1].shape)

In [ ]:
import cPickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('exp/neural-networks-and-deep-learning-master//data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()

In [ ]:
len(training_data)
training_data[0][0].shape

In [ ]:
z = weights[0].dot(training_data[0][0])

In [ ]:
z.shape

In [ ]:
z = z + biases[0]

In [ ]:
z.shape

In [ ]:
b